In [1]:
from selenium import webdriver

from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import pandas as pd
import datetime
import time
import os
import requests

In [2]:
import praw
# Log In to App: 
reddit = praw.Reddit(client_id='-1G_AVJTJzG_ILhPlC0r1Q', client_secret='HBAm1HycBjYjvBoUMyFoTDe5opda_g', user_agent='datainneed by u/haizcestlavie')

Version 7.4.0 of praw is outdated. Version 7.7.1 was released Tuesday July 11, 2023.


In [3]:
def get_subreddit_submissions(reddit, subreddit_name, text):
    # Get the subreddit new posts
    subreddit = reddit.subreddit(subreddit_name).new(limit=1000)

    data = []
    # Search for the text in the subreddit's submissions
    for submission in subreddit:
        # Convert the title to lowercase and check if it contains the text
        if text.lower() in submission.title.lower():
            data.append({
                'title': submission.title,
                'num_comments': submission.num_comments,
                'url': submission.url,
                'created': datetime.datetime.fromtimestamp(submission.created),
                'upvotes': submission.ups,
                'id': submission.id,
                'score': submission.score #score is upvotes - downvotes
            }) 

    # Convert the list of dictionaries into a DataFrame
    df = pd.DataFrame(data)
    return df

In [4]:
def get_comments(reddit, df):
    # Get the list of submission IDs from the DataFrame
    submission_ids = df['id'].tolist()

    data = []

    # Loop over the submission IDs
    for submission_id in submission_ids:
        # Get the submission
        submission = reddit.submission(id=submission_id)
        # Get the comments of the submission
        submission.comments.replace_more(limit=None)  # This line expands the comment forest if a "load more comments" placeholder is encountered
        for comment in submission.comments.list():
            data.append({
                'submission_id': submission_id,
                'submission_title': submission.title,
                'date': datetime.datetime.fromtimestamp(comment.created),
                'comment_text': comment.body,
                'upvotes': comment.ups,
            })

    # Convert the list of dictionaries into a DataFrame
    df_comments = pd.DataFrame(data)
    return df_comments

# Stock code TSLA from subreddit r/wallstreetbets

In [5]:
wallstreetbets_df_tsla = get_subreddit_submissions(reddit, 'wallstreetbets', 'tsla')
wallstreetbets_df_tsla_comments = get_comments(reddit, wallstreetbets_df_tsla)
wallstreetbets_df_tsla_comments.to_csv('wallstreetbets_tsla_comments2.csv', index=False)
wallstreetbets_df_tsla_comments

,submission_id,submission_title,date,comment_text,upvotes
0,1aor79h,Thoughts on TSLA tomorrow?,2024-02-12 11:15:36,\n**User Report**| | | |\n:--|:--|:--|:--\n**T...,1
1,1aor79h,Thoughts on TSLA tomorrow?,2024-02-12 13:40:09,tomorrow elon going to pump tesla through X,16
2,1aor79h,Thoughts on TSLA tomorrow?,2024-02-12 12:46:30,Elaborate you fuck,28
3,1aor79h,Thoughts on TSLA tomorrow?,2024-02-12 12:19:49,Calls.,24
4,1aor79h,Thoughts on TSLA tomorrow?,2024-02-12 12:02:39,What’s going on with Tesla tomorrow??,10
...,...,...,...,...,...
260,1ai8yu9,TSLA 35k 192.50 calls expiring 02/09. Make or ...,2024-02-04 23:00:07,"Not that regarded, God bless",1
261,1ai8yu9,TSLA 35k 192.50 calls expiring 02/09. Make or ...,2024-02-04 09:32:17,I guess a decent chunk is all relative. Sorry ...,2
262,1ai8yu9,TSLA 35k 192.50 calls expiring 02/09. Make or ...,2024-02-05 08:06:51,I trust you bro,1
263,1ai8yu9,TSLA 35k 192.50 calls expiring 02/09. Make or ...,2024-02-04 09:39:47,I bought 100k worth the other day and made a d...,4


# Stock code TSLA from subreddit r/stocks

In [6]:
stocks_df_tsla = get_subreddit_submissions(reddit, 'stocks', 'tsla')
stocks_df_tsla_comments = get_comments(reddit, stocks_df_tsla)
stocks_df_tsla_comments.to_csv('stocks_tsla_comments2.csv', index=False)
stocks_df_tsla_comments

,submission_id,submission_title,date,comment_text,upvotes
0,19fjwcz,TSLA for Long Term,2024-01-26 07:58:26,I wouldn’t take reddits advice on anything TSL...,26
1,19fjwcz,TSLA for Long Term,2024-01-26 04:24:49,The easy answer is that Tesla will be a very s...,79
2,19fjwcz,TSLA for Long Term,2024-01-26 04:30:37,Was bullish but no longer. I sold everything a...,47
3,19fjwcz,TSLA for Long Term,2024-01-26 12:41:39,"I love Tesla, and I like Elon. I held the stoc...",5
4,19fjwcz,TSLA for Long Term,2024-01-26 04:28:57,As a company- Bearish.. Chinese automakers wil...,8
...,...,...,...,...,...
723,18s30mt,"Whats your ""this could be the next tsla or aap...",2023-12-28 03:48:44,"Hey, you're the one who doesn't know that 52 i...",1
724,18s30mt,"Whats your ""this could be the next tsla or aap...",2023-12-28 03:54:06,lol so you know how to pick stocks that are go...,1
725,18s30mt,"Whats your ""this could be the next tsla or aap...",2023-12-28 03:58:08,"Hey idk if you've realized this yet, but maybe...",1
726,18s30mt,"Whats your ""this could be the next tsla or aap...",2023-12-28 04:05:54,Oh wow you don’t need stop losses if you buy s...,1


# Stock code TSLA from subreddit r/StockMarket

In [7]:
stockmarket_df_tsla = get_subreddit_submissions(reddit, 'StockMarket', 'tsla')
stockmarket_df_tsla_comments = get_comments(reddit, stockmarket_df_tsla)
stockmarket_df_tsla_comments.to_csv('stockmarket_tsla_comments2.csv', index=False)
stockmarket_df_tsla_comments

,submission_id,submission_title,date,comment_text,upvotes
0,1abv5ee,Routine Analysis of TSLA,2024-01-27 06:48:36,This is such a generic and pointless post.,27
1,1abv5ee,Routine Analysis of TSLA,2024-01-27 19:02:34,thanks for summarizing yahoo finance I guess?,6
2,1abv5ee,Routine Analysis of TSLA,2024-01-27 06:25:13,Tesla is going to fall to between $100 & $150.,-8
3,1abv5ee,Routine Analysis of TSLA,2024-01-28 05:04:35,How are they long-term resilient if they're do...,1
4,1abv5ee,Routine Analysis of TSLA,2024-01-27 07:20:17,Welcome to r/stockmarket.\n\nIt's all just bot...,6
...,...,...,...,...,...
350,17rjdtp,TSLA dips 6%!,2023-11-10 06:40:29,"Find another car stock that has energy, roboti...",5
351,17rjdtp,TSLA dips 6%!,2023-11-10 08:28:36,Tesla gets their batteries from Panasonic. Ano...,10
352,17rjdtp,TSLA dips 6%!,2023-11-10 09:55:54,Lol,1
353,17rjdtp,TSLA dips 6%!,2023-11-10 13:15:30,But they will sell billions of sex robots!! /s,5
